In [1]:
import numpy as np
import torch
import torch.nn as nn
import importlib as imp
np.random.seed(0)
torch.manual_seed(0)

In [2]:
a = np.asarray(list(range(10)))
np.random.shuffle(a)
a

array([2, 8, 4, 9, 1, 6, 7, 3, 0, 5])

In [35]:
import data
imp.reload(data)
manager = data.Meme_n_Caption_Data_Manager(
    FILENAME_all_image_embeddings='image_embeddings.pt',
    FILENAME_all_caption_embeddings='caption_embeddings.pt',
    FILENAME_MAP_image_index_TO_caption_indices='image_index_TO_caption_index.npy'
)

Device: METAL
Device: METAL


In [32]:
all_indices = list(range(len(manager.MAP_image_index_TO_caption_indices[0])))
np.random.shuffle(all_indices)
train_ratio, validate_ratio, test_ratio = 0.7, 0.1, 0.2
assert train_ratio + validate_ratio + test_ratio == 1, "STOP! Train/Validate/Test split NO add to 1"
test_indices = np.asarray(all_indices[int(len(all_indices) * (1 - test_ratio)):])
train_and_validate_indices = np.asarray(all_indices[:int(len(all_indices) * (1 - test_ratio))])
assert len(train_and_validate_indices) + len(test_indices) == len(all_indices), "STOP! T/V/T indices split wrong"
# train_indices = all_indices[:int(len(all_indices) * (train_ratio))]
# validate_indices = all_indices[int(len(all_indices) * (train_ratio)):int(len(all_indices) * (1 - test_ratio))]
# assert len(train_indices) + len(validate_indices) + len(test_indices) == len(all_indices), "STOP! T/V/T indices split wrong"

In [23]:
from sklearn.model_selection import KFold
DOER_CV = KFold(n_splits=5, shuffle=True, random_state=0)
indices = np.arange(13)
i = None
for t1, t2 in DOER_CV.split(np.asarray(train_and_validate_indices)[:, None], train_and_validate_indices):
    print(len(t1), len(t2))
    i = t1
    break

1920 480


In [31]:
type(train_and_validate_indices)

list

In [25]:
dl = manager.get_dataloader(batch_size=10, RATIO_negative_samples_over_positive_samples=1, INDICES_usable_captions=i)

In [28]:
for b in dl:
    a, b, c = b
    print(a.shape, b.shape, c.shape)
    break

torch.Size([10, 2048]) torch.Size([10, 384]) torch.Size([10])


In [36]:
from numpy.typing import NDArray
import matplotlib.pyplot as plt
import model
imp.reload(model)
def cross_validate(m: nn.Module, hyperparameters,
        plot: bool=False):
    DOER_CV = KFold(n_splits=5, shuffle=True, random_state=0)
    for train_indicesCEPTION, validate_indicesCEPTION in DOER_CV.split(
        np.asarray(train_and_validate_indices)[:, None], train_and_validate_indices):
        # training
        train_losses = model.train(model=m, 
            dataloader_train=manager.get_dataloader(
                batch_size=hyperparameters['batch_size'], 
                RATIO_negative_samples_over_positive_samples=hyperparameters['ratio_-_samples_count_over_+_samples_count'], 
                INDICES_usable_captions=train_and_validate_indices[train_indicesCEPTION]), hyperparameters=hyperparameters)
        if plot:
            f, ax = plt.subplots(figsize=(4, 4))
            ax.plot(train_losses)
        break
    return m
    # inference
    test_losses, test_accuracy, aurocs, auprcs = model.test(model=m, dataloader_test=data.get_dataloader(
        hyperparameters=hyperparameters,
        selected_features=features_test,
        selected_labels=labels_test))
    auroc_mean, auroc_std = np.asarray(aurocs).mean(), np.asarray(aurocs).std()
    auprc_mean, auprc_std = np.asarray(auprcs).mean(), np.asarray(auprcs).std()
    losses_mean, losses_std = np.asarray(test_losses).mean(), np.asarray(test_losses).std()
    # print(m)
    print(f"Test AUROC: {auroc_mean, auroc_std}")
    print(f"Test AUPRC: {auprc_mean, auprc_std}")
    print(f"Test Loss: {losses_mean, losses_std}")
    print(f"Test Accuracy: %{test_accuracy}")
    return m

hyperparameters = {'learning_rate': 1e-3, 'batch_size': 64, 
    'optimizer_type': torch.optim.Adam, 'ratio_-_samples_count_over_+_samples_count': 1.0}
m = cross_validate(model.Siamese(), hyperparameters,
    plot=True)

Device: METAL


RuntimeError: Invalid buffer size: 8.79 GB